In [1]:
import math
import scipy
import numpy as np
import pandas as pd
import math as m
from statsmodels.tsa.stattools import acf
from statsmodels.graphics.tsaplots import plot_acf

from scipy.spatial import distance_matrix
from scipy import linalg
from scipy.optimize import minimize
from scipy.linalg import eig, eigh
from scipy.fft import fft
from scipy.special import eval_gegenbauer, sph_harm

from sklearn import svm, datasets
from sklearn.datasets import make_swiss_roll
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import kneighbors_graph, NearestNeighbors
from sklearn.decomposition import KernelPCA, PCA
from sklearn.manifold import TSNE
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#import scaleogram as scg
from matplotlib import pyplot as plt
from matplotlib import cm

import warnings

from tqdm.notebook import tqdm

from visuals import *
from my_lib import *
from SSA_lib import SSA 

In [2]:
import warnings
warnings.simplefilter('ignore')

In [3]:
def CartesianToSpherical(point):
    r = np.sqrt(sum(point ** 2))
    n = len(point)
    phi = np.zeros(n - 1)
    
    for i in range(n - 2):
        phi[i] = np.arccos(point[i] / np.sqrt(sum(point[i:] ** 2)))
        
    if point[-1] >= 0:
        phi[n - 2] = np.arccos(point[n - 2] / np.sqrt(point[n - 1] ** 2 + point[n - 2] ** 2))
    else:
        phi[n - 2] = 2 * np.pi - np.arccos(point[n - 2] / np.sqrt(point[n - 1] ** 2 + point[n - 2] ** 2))
        
    return np.hstack((phi, r))


def TrajectoryToSpherical(tr):
    tr_spherical = np.zeros(tr.shape)
    for i, point in enumerate(tr):
        tr_spherical[i] = CartesianToSpherical(point)
    return tr_spherical


def SphericalToCartesian(point):
    phi, r = point[:-1], point[-1]
    n = len(point)
    x = np.zeros(n) 
    cur = r
    
    for i in range(n - 1):
        x[i] = cur * np.cos(phi[i])
        cur *= np.sin(phi[i])

    x[n - 1] = cur
    return x
    
def TrajectoryToCartesian(tr):
    tr_cartesian = np.zeros(tr.shape)
    for i, point in enumerate(tr):
        tr_cartesian[i] = SphericalToCartesian(point)
    return tr_cartesian

def HankelMatrix(X, L):  
    N = X.shape[0]
    return scipy.linalg.hankel(X[ : N - L + 1], X[N - L : N])

In [4]:
from scipy.special import lpmv, gamma, hyp2f1

def k_c_l_L(k,l,L):
    """ Normalizing  constant """
#     print('k,l,L', k,l,L)
#     print('L + l + k - 2', L + l + k - 2)
#     print('L - l', L - l)
    ans = ((2*L + k - 1)/ 2) \
    * math.factorial(L + l + k - 2)/math.factorial(L - l)
    ans = ans**.5
    return ans

def P_neg_mu_nu(neg_mu,nu,x):
    """The associated Legendre function"""
    mu = - neg_mu
    return (1/gamma(1+mu)) * (((1-x)/(1+x))**(mu/2)) * (hyp2f1(-nu,nu+1,1+mu,(1-x)/2))

def hat_k_P_L(k, l, L, alpha):
    hat_k_P_L_ans = k_c_l_L(k,l,L)
    hat_k_P_L_ans *= (np.sin(alpha)) ** (-(k-2)/2)
    hat_k_P_L_ans *= P_neg_mu_nu(
        -(l + ((k-2)/2)),
        +(L + ((k-2)/2)),
        np.cos(alpha)
    )
    return hat_k_P_L_ans

def high_order_sph_harm(l, theta, phi):
    k = len(l)
    high_order_sph_harm_array = np.exp(1j*l[0]*phi).reshape((-1,1))
    
    for i in range(k-1):
        high_order_sph_harm_array *= np.nan_to_num(
            hat_k_P_L(
                int(2+i),
                np.abs(l[i]),
                l[i+1],
                theta[:,i]) * (1/((2*np.pi)**.5)),
            nan = 0.0).reshape((-1,1))
        
    if l[0] > 0:
        return (-1)**l[0] * high_order_sph_harm_array
    else:
        return high_order_sph_harm_array

In [5]:
def HSH_deg3(step_X_full, N_degree):
    sph_harm_discret_basis = []

    for l_2 in np.arange(N_degree):
        for l_1 in np.arange(l_2+1):
            for l_0 in range(-l_1,l_1+1):
                if l_0 >= 0:
                    sph_harm_discret_basis.append(
                        np.real(
                            high_order_sph_harm([l_0,l_1,l_2], step_X_full[:,:-1], step_X_full[:,-1])
                        )
                    )
                elif l_0 < 0:
                    sph_harm_discret_basis.append(
                        np.imag(
                            high_order_sph_harm([l_0,l_1,l_2], step_X_full[:,:-1], step_X_full[:,-1])
                        )
                    )

    return np.array(sph_harm_discret_basis).T[0]

In [6]:
GRAND_RESULT = {}
models_dict = {}

In [7]:
history = {}

In [8]:
class_motion = 'ups_12'
dir_list = ['wlk_8', 'jog_9', 'ups_12']
sub_num = 1
min_dim = 4


# обучили PCA для конкретного движения
df = pd.read_csv(f'./data/A_DeviceMotion_data/{class_motion}/sub_{sub_num}.csv')
x_1 = df[['gravity.x', 'gravity.y','gravity.z']].to_numpy()*9.8
x_2 = df[['userAcceleration.x', 'userAcceleration.y','userAcceleration.z']].to_numpy()*9.8
x = x_1+x_2
x = np.sum(x**2,axis = 1)**.5 - 9.8
pivot = int(1/3 * len(x))
print(pivot)
X = HankelMatrix(x[:pivot], 100)
pca_ = PCA(n_components = min_dim)
pca_.fit(X)
models_dict[f'PCA_{class_motion}_dim{min_dim}'] = pca_

X_collection = []
y_collection = []

for i, motion in enumerate(dir_list):
    df = pd.read_csv(f'./data/A_DeviceMotion_data/{motion}/sub_{sub_num}.csv')
    x_1 = df[['gravity.x', 'gravity.y','gravity.z']].to_numpy()*9.8
    x_2 = df[['userAcceleration.x', 'userAcceleration.y','userAcceleration.z']].to_numpy()*9.8
    x = x_1+x_2
    x = np.sum(x**2,axis = 1)**.5 - 9.8
    pivot = int(2/3 * len(x))
    
    motion_X = HankelMatrix(x[:pivot], 100)
    motion_X = models_dict['PCA_'+str(class_motion)+'_dim'+str(min_dim)].transform(motion_X)
    for subsample_size in range(0,int(motion_X.shape[0]-300),25):
        X_collection.append(motion_X[subsample_size:subsample_size+300])
        if motion == class_motion:
            y_collection.append(1)
        else:
            y_collection.append(0)
        
y_collection = np.array(y_collection)

355


In [9]:
phi = np.linspace(0, 2*np.pi, 30)
theta = np.linspace(0, np.pi, 30)
phi, theta_1, theta_2 = np.meshgrid(phi, theta, theta)
phi = phi.reshape(-1,1)
theta_1 = theta_1.reshape(-1,1)
theta_2 = theta_2.reshape(-1,1)
theta = np.concatenate((theta_2, theta_1, phi), axis = 1)
Y_0 = np.zeros((len(theta),1))

SH_basis_init = HSH_deg3(theta, N_degree = 5)

Y_0 = np.zeros((len(theta),1))

w_description = []

for i in tqdm(np.arange(0,len(X_collection))):

    Y_1 = np.ones((X_collection[i].shape[0],1))
    step_X_sp = TrajectoryToSpherical(X_collection[i])[:,:-1]
    step_Y_full = np.concatenate((Y_0, Y_1), axis = 0)

    SH_basis_data = HSH_deg3(step_X_sp, N_degree = 5)

    SH_basis_full = np.concatenate((SH_basis_init, SH_basis_data), axis = 0)

    clf = LogisticRegression(penalty = 'elasticnet',
                             l1_ratio = 0.5,
                             random_state = 42,
                             fit_intercept = False,
                             solver='saga',
                             class_weight = 'balanced'
                            )
    clf.fit(SH_basis_full, step_Y_full)

    w_description.append(clf.coef_[0])
        
w_description = np.array(w_description)

  0%|          | 0/227 [00:00<?, ?it/s]

In [10]:
clf = svm.SVC(kernel='rbf').fit(w_description, y_collection)
models_dict[f'SVM_{class_motion}_dim{min_dim}'] = clf

In [11]:
models_dict

{'PCA_ups_12_dim4': PCA(n_components=4), 'SVM_ups_12_dim4': SVC()}

In [12]:
X_collection_test = []
y_collection_test = []
sub_num = 1
for i, motion in enumerate(dir_list):
    df = pd.read_csv(f'./data/A_DeviceMotion_data/{motion}/sub_{sub_num}.csv')
    x_1 = df[['gravity.x', 'gravity.y','gravity.z']].to_numpy()*9.8
    x_2 = df[['userAcceleration.x', 'userAcceleration.y','userAcceleration.z']].to_numpy()*9.8
    x = x_1+x_2
    x = np.sum(x**2,axis = 1)**.5 - 9.8
    pivot = int(2/3 * len(x))
    
    motion_X = HankelMatrix(x[pivot:], 100)
    motion_X = models_dict['PCA_'+str(class_motion)+'_dim'+str(min_dim)].transform(motion_X)
    for subsample_size in range(0,int(motion_X.shape[0]-300),25):
        X_collection_test.append(motion_X[subsample_size:subsample_size+300])
        if motion == class_motion:
            y_collection_test.append(1)
        else:
            y_collection_test.append(0)
        
y_collection_test = np.array(y_collection_test)

In [13]:
w_description_test = []

for i in tqdm(np.arange(0,len(X_collection_test))):

    Y_1 = np.ones((X_collection[i].shape[0],1))
    step_X_sp = TrajectoryToSpherical(X_collection_test[i])[:,:-1]
    step_Y_full = np.concatenate((Y_0, Y_1), axis = 0)

    SH_basis_data = HSH_deg3(step_X_sp, N_degree = 5)

    SH_basis_full = np.concatenate((SH_basis_init, SH_basis_data), axis = 0)

    clf = LogisticRegression(penalty = 'elasticnet',
                             l1_ratio = 0.5,
                             random_state = 42,
                             fit_intercept = False,
                             solver='saga',
                             class_weight = 'balanced',
#                              max_iter = 300
                            )
    clf.fit(SH_basis_full, step_Y_full)

    w_description_test.append(clf.coef_[0])
        
w_description_test = np.array(w_description_test)

  0%|          | 0/91 [00:00<?, ?it/s]

In [14]:
clf_pred = models_dict[f'SVM_{class_motion}_dim{min_dim}'].predict(w_description)
print(f1_score(y_collection, clf_pred))

1.0


In [15]:
clf_pred = models_dict[f'SVM_{class_motion}_dim{min_dim}'].predict(w_description_test)
f1_score(y_collection_test, clf_pred)

0.0

In [16]:
w_description_subs = []
y_collection_subs = []

for sub_num in tqdm([1,9,22]):

    X_collection_sub = []
    y_collection_sub = []
    

    for i, motion in enumerate(dir_list):
        df = pd.read_csv(f'./data/A_DeviceMotion_data/{motion}/sub_{sub_num}.csv')
        x_1 = df[['gravity.x', 'gravity.y','gravity.z']].to_numpy()*9.8
        x_2 = df[['userAcceleration.x', 'userAcceleration.y','userAcceleration.z']].to_numpy()*9.8
        x = x_1+x_2
        x = np.sum(x**2,axis = 1)**.5 - 9.8

        motion_X = HankelMatrix(x[:1000], 100)
        motion_X = models_dict['PCA_'+str(class_motion)+'_dim'+str(min_dim)].transform(motion_X)
#         fig = plt.figure(figsize=(5, 5))
#         ax = fig.add_subplot(111,projection='3d')
#         ax.plot(motion_X[:1000,0], motion_X[:1000,1], motion_X[:1000,2], lw = 3)
#         plt.show()
        
        
        for subsample_size in range(0,int(motion_X.shape[0]-400),25):
            X_collection_sub.append(motion_X[subsample_size:subsample_size+400])
            if motion == class_motion:
                y_collection_sub.append(1)
            else:
                y_collection_sub.append(0)
        
        

    y_collection_sub = np.array(y_collection_sub)
    w_description_sub = []

    for i in tqdm(np.arange(0,len(X_collection_sub)), leave = False):

        Y_1 = np.ones((X_collection_sub[i].shape[0],1))
        step_X_sp = TrajectoryToSpherical(X_collection_sub[i])[:,:-1]
        step_Y_full = np.concatenate((Y_0, Y_1), axis = 0)

        SH_basis_data = HSH_deg3(step_X_sp, N_degree = 5)
        
        SH_basis_full = np.concatenate((SH_basis_init, SH_basis_data), axis = 0)

        clf = LogisticRegression(penalty = 'elasticnet',
                                 l1_ratio = 0.5,
                                 random_state = 42,
                                 fit_intercept = False,
                                 solver='saga',
                                 class_weight = 'balanced',
#                                  max_iter = 300
                                )
        clf.fit(SH_basis_full, step_Y_full)

        w_description_sub.append(clf.coef_[0])

    w_description_sub = np.array(w_description_sub)
    
    w_description_subs.append(w_description_sub)
    y_collection_subs.append(y_collection_sub)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

  0%|          | 0/63 [00:00<?, ?it/s]

In [17]:
ans = 0
for i in range(len(w_description_subs)):
    clf_pred = models_dict[f'SVM_{class_motion}_dim{min_dim}'].predict(w_description_subs[i])
    history[f'{class_motion}_{i}'] = f1_score(y_collection_subs[i], clf_pred)
    print(f1_score(y_collection_subs[i], clf_pred))
    ans += f1_score(y_collection_subs[i], clf_pred)
ans /= len(w_description_subs)
print()
print(ans)

0.6875000000000001
0.0
0.0

0.2291666666666667


In [18]:
history

{'ups_12_0': 0.6875000000000001, 'ups_12_1': 0.0, 'ups_12_2': 0.0}

In [19]:
GRAND_RESULT[class_motion] = ans

In [ ]:
{'wlk_8_0': 1.0,
 'wlk_8_1': 1.0,
 'wlk_8_2': 0.9333333333333333,
 'jog_9_0': 1.0,
 'jog_9_1': 1.0,
 'jog_9_2': 0.32,
 'ups_12_0': 0.6451612903225806,
 'ups_12_1': 0.3846153846153846,
 'ups_12_2': 0.25}

In [ ]:
GRAND_RESULT

In [ ]:
np.mean([0.9848484848484849, 0.8148148148148149, 0.6451612903225806])

In [ ]:
np.std([0.9848484848484849, 0.8148148148148149, 0.6451612903225806])